In [38]:
from pymongo import MongoClient
from metadataclient.api import (insert_run_start, insert_descriptor,
                                insert_run_stop, insert_event, bulk_insert_events)
from metadataclient.api import (find_run_starts, find_descriptors, find_events,
                               find_run_stops, find_last)
from metadataclient.conf import connection_config
from urllib3.exceptions import ConnectionError
import boltons

In [16]:
class MdsMigrationException(Exception):
    """Goodbye cruel world!"""
    pass

**_Ensure global connection management is functional_**

In [17]:
try:
    MONGO_HOST = 'kusadasi'
    connection_config['host'] = MONGO_HOST 
    res = next(find_last())
except Exception:
    pass

**Connection Management**

In [34]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
# assuming both mongo and tornado server on the same machine
connection_config['host'] = MONGO_HOST 
connection_config['port'] = 7770
pymongo_client = MongoClient(MONGO_HOST, MONGO_PORT)
database = pymongo_client[MIGRATION_DB]
try:
    res = (next(find_last()))
except Exception:
    raise MdsMigrationException('Seems like metadataservice is not accessible')

**Get all run_starts**

In [35]:
print(database)

rstart_crsr = database.run_start.find()

Database(MongoClient('localhost', 27017), 'datastore2')


In [36]:
run_starts = {}
for rstart in rstart_crsr:
    run_starts[rstart['uid']] = rstart
    bcfg_crsr = database.beamline_config.find()
    for cfg in bcfg_crsr:
        rstart['config'] = cfg

**Get all descriptors and insert them using metadataclient**

In [39]:
desc_crsr = database.event_descriptor.find()

In [1]:
def rstart_oid_lookup(database, oid):
    return next(database.run_start.find({'_id':oid}))

In [ ]:
for desc in desc_crsr:
    rs_oid = desc['run_start']
    rstart = rstart_oid_lookup(database, rs_oid)
    desc['run_start'] = run_starts[rstart['uid']]['uid'] #make sure run_start exists